# Detector Hits in IceCube-86

Demonstrate detector hits in the IC86 configuration of IceCube.

In [ ]:
from astropy import units as u

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from snewpy.neutrino import Flavor

from asteria.simulation import Simulation
from asteria import set_rcparams
from asteria import interactions

set_rcparams(verbose=False)

## Load a SNEWPY Model and Configure the Simulation

Set up a progenitor 10 kpc from Earth and generate simulated hits.

In [ ]:
model = {'name': 'Nakazato_2013',
         'param':{
             'progenitor_mass': 13 * u.Msun,
             'revival_time': 300 * u.ms,
             'metallicity': 0.004,
             'eos': 'shen'}
         }

sim = Simulation(model=model,
                 distance=10 * u.kpc, 
                 Emin=0*u.MeV, Emax=100*u.MeV, dE=1*u.MeV,
                 tmin=-1*u.s, tmax=1*u.s, dt=1*u.ms)
sim.run()

## Plot the Energy Deposit

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6), tight_layout=True)

for flavor in sim.flavors:
    ax.plot(sim.time, sim.E_per_V[flavor], label=flavor.to_tex())
ax.legend()
ax.set(xlabel=r'$t-t_\mathrm{bounce}$ [s]',
       ylabel='energy deposit [Mev m$^{-3}$])',
       xlim=(-0.15, 0.75));

## Plot Detector Response

### Expected Signal from Each Subdetector

Set a time resolution `dt`. Using the `sim.detector_signal()` function we can read out the detector signal for each subdetector class. Separately plot hits from the main IceCube strings and DeepCore.

In [ ]:
dt = 2 * u.ms
t, sim_i3 = sim.detector_signal(subdetector='i3', dt=dt)
t, sim_dc = sim.detector_signal(subdetector='dc', dt=dt)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6), tight_layout=True)
ax.plot(t, sim_i3, label='IceCube DOM')
ax.plot(t, sim_dc, label='HQE DOM (DeepCore)')
ax.legend()
ax.set(xlabel=r'$t-t_\mathrm{bounce}$ [s]',
       ylabel=f'detector hits',
       xlim=(-0.15, 0.75),
       ylim=(0,2000));

### Generated Hits from Signal Only

In [ ]:
t, hits_i3 = sim.detector_hits(subdetector='i3', dt=dt)
t, hits_dc = sim.detector_hits(subdetector='dc', dt=dt)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6), tight_layout=True)
ax.plot(t, hits_i3, label='IceCube DOM')
ax.plot(t, hits_dc, label='HQE DOM (DeepCore)')
ax.legend()
ax.set(xlabel=r'$t-t_\mathrm{bounce}$ [s]',
       ylabel=f'detector hits',
       xlim=(-0.15, 0.75),
       ylim=(0,2000));

### Generated Hits from Signal + Background

Separately compute the background hits and signal from each subdetector and add them.

In [ ]:
bkg_i3 = sim.detector.i3_bg(dt, size=hits_i3.size)
bkg_dc = sim.detector.dc_bg(dt, size=hits_dc.size)
bkg = bkg_i3 + bkg_dc
hits = hits_i3 + hits_dc

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6), tight_layout=True)
ax.plot(t, hits_i3 + bkg_i3, label='IceCube DOM')
ax.plot(t, hits_dc + bkg_dc, label='HQE DOM (DeepCore)')
ax.plot(t, hits + bkg, label='Total hits')
ax.legend(loc='upper right', fontsize=14)
ax.set(xlabel=r'$t-t_\mathrm{bounce}$ [s]',
       ylabel=f'detector hits',
       xlim=(-0.15, 0.75),
       ylim=(0,6000));